In [1]:
import ray
ray.init(address="auto", ignore_reinit_error=True)

2020-12-01 17:19:36,070	INFO worker.py:634 -- Connecting to existing Ray cluster at address: 192.168.8.51:6379


{'node_ip_address': '192.168.8.51',
 'raylet_ip_address': '192.168.8.51',
 'redis_address': '192.168.8.51:6379',
 'object_store_address': '/tmp/ray/session_2020-12-01_15-12-48_169152_13834/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-12-01_15-12-48_169152_13834/sockets/raylet',
 'webui_url': '192.168.8.51:8265',
 'session_dir': '/tmp/ray/session_2020-12-01_15-12-48_169152_13834',
 'metrics_export_port': 60913}

## Load cifar to define witch images to train with

In [2]:
# execute only if run as a script
import random

#############################################################################################
# LOAD DATA
#############################################################################################
from data_utils import CIFAR10Data
# Load data
cifar10_data = CIFAR10Data()
num_classes = len(cifar10_data.classes)
x_train, y_train, x_test, y_test = cifar10_data.get_data(subtract_mean=True)

#############################################################################################
# LOAD CONFIG
#############################################################################################
from config import Load_config
config = Load_config(len(x_train),x_train.shape[1:],num_classes,active_learning = False)

# Initialize a labeled dataset by randomly sampling K=ADDENDUM=1,000 data points from the entire dataset.
indices = list(range(config["NUM_TRAIN"]))
random.shuffle(indices)
labeled_set = indices[:config["ADDENDUM"]]
unlabeled_set = indices[config["ADDENDUM"]:]

/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/reminiz/ReminizML2/python_env/lib/pyt

CIFAR10 Training data shape: (50000, 32, 32, 3)
CIFAR10 Training label shape (50000, 1)
CIFAR10 Test data shape (10000, 32, 32, 3)
CIFAR10 Test label shape (10000, 1)


## Import the training module

In [3]:
from train_agent_cifar import AL_train_cifar

## Create a Training Actor with wandb project name, dataset source, dataset name, configuration and batch size as parameters respectively

In [4]:
#project, source, dataset_name, config, val_source=None, val_dataset_name=None, bs=32, agnostic_eval=False)
NetworkActor =  AL_train_cifar.remote(config, labeled_set)

(pid=13696) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=13696)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=13696) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=13696)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=13696) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=13696)   _np_qint16 = np.dtyp

In [5]:
NetworkActor

(pid=13696) CIFAR10 Training data shape: (50000, 32, 32, 3)
(pid=13696) CIFAR10 Training label shape (50000, 1)
(pid=13696) CIFAR10 Test data shape (10000, 32, 32, 3)
(pid=13696) CIFAR10 Test label shape (10000, 1)


Actor(AL_train_cifar,dbc81ea81c000000)

(pid=13696) WARNING:tensorflow:From /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:429: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
(pid=13696) Instructions for updating:
(pid=13696) tf.py_func is deprecated in TF V2. Instead, use
(pid=13696)     tf.py_function, which takes a python function which manipulates tf eager
(pid=13696)     tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
(pid=13696)     an ndarray (just call tensor.numpy()) but having access to eager tensors
(pid=13696)     means `tf.py_function`s can use accelerators such as GPUs as well as
(pid=13696)     being differentiable using a gradient tape.
(pid=13696)     
(pid=13696) WARNING:tensorflow:From /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated a

## Check if the training is active

In [6]:
progress_id = NetworkActor.isTraining.remote()
ray.get(progress_id)

(pid=13696) 2020-12-01 17:20:15.488450: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=13696) 2020-12-01 17:20:15.512847: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3300280000 Hz
(pid=13696) 2020-12-01 17:20:15.513667: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x992e060 executing computations on platform Host. Devices:
(pid=13696) 2020-12-01 17:20:15.513691: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
(pid=13696) 2020-12-01 17:20:15.671739: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x996a300 executing computations on platform CUDA. Devices:
(pid=13696) 2020-12-01 17:20:15.671777: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): TITAN RTX, Compute Capability 7.5
(pid=13696) 2020-12-01 17:20:15.672037: I tensorflow/core/common_r

False

(pid=13696) 2020-12-01 17:20:18,016	INFO worker.py:634 -- Connecting to existing Ray cluster at address: 192.168.8.51:6379
(pid=13696) 2020-12-01 17:20:18,016	ERROR worker.py:643 -- Calling ray.init() again after it has already been called.


## Start the training

In [7]:
NetworkActor.start_training.remote()

(pid=13696) => Starts to train from scratch 
(pid=13696) self.steps_per_epoch 391


(pid=13696) 2020-12-01 17:20:23.768386: I tensorflow/stream_executor/dso_loader.cc:152] successfully opened CUDA library libcublas.so.10.0 locally


(pid=13696) Epoch:  1 / 51 || Time: 2020-12-01 17:21:14 || Accuracy: 34.90 || MAE LossNet: 3.32 || 


(pid=2699, ip=192.168.8.55) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=2699, ip=192.168.8.55)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=2699, ip=192.168.8.55) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=2699, ip=192.168.8.55)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=2699, ip=192.168.8.55) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it wil

(pid=2699, ip=192.168.8.55) CIFAR10 Training data shape: (50000, 32, 32, 3)
(pid=2699, ip=192.168.8.55) CIFAR10 Training label shape (50000, 1)
(pid=2699, ip=192.168.8.55) CIFAR10 Test data shape (10000, 32, 32, 3)
(pid=2699, ip=192.168.8.55) CIFAR10 Test label shape (10000, 1)


...

 89% 927/1037 [01:35<00:13,  8.42it/s] 
 90% 929/1037 [01:35<00:11,  9.11it/s] 
 90% 931/1037 [01:35<00:11,  9.60it/s] 
 90% 933/1037 [01:35<00:10,  9.62it/s] 
 90% 935/1037 [01:36<00:10,  9.63it/s] 
 90% 936/1037 [01:36<00:10,  9.39it/s] 
 90% 937/1037 [01:36<00:10,  9.15it/s] 
 91% 939/1037 [01:36<00:10,  9.57it/s] 
 91% 941/1037 [01:36<00:10,  9.54it/s] 
 91% 943/1037 [01:36<00:09,  9.56it/s] 
 91% 945/1037 [01:37<00:09,  9.75it/s] 
 91% 947/1037 [01:37<00:09,  9.91it/s] 
 92% 949/1037 [01:37<00:08, 10.19it/s] 
(pid=7715, ip=192.168.8.55) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=7715, ip=192.168.8.55)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=7715, ip=192.168.8.55) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/pyth

(pid=7715, ip=192.168.8.55) CIFAR10 Training data shape: (50000, 32, 32, 3)
(pid=7715, ip=192.168.8.55) CIFAR10 Training label shape (50000, 1)
(pid=7715, ip=192.168.8.55) CIFAR10 Test data shape (10000, 32, 32, 3)
(pid=7715, ip=192.168.8.55) CIFAR10 Test label shape (10000, 1)


 95% 986/1037 [01:41<00:05,  8.81it/s] 
 95% 988/1037 [01:41<00:05,  9.36it/s] 
 95% 990/1037 [01:41<00:04,  9.62it/s] 
 96% 992/1037 [01:42<00:04,  9.79it/s] 
 96% 994/1037 [01:42<00:04, 10.04it/s] 
(pid=7715, ip=192.168.8.55) WARNING:tensorflow:From /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:429: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
(pid=7715, ip=192.168.8.55) Instructions for updating:
(pid=7715, ip=192.168.8.55) tf.py_func is deprecated in TF V2. Instead, use
(pid=7715, ip=192.168.8.55)     tf.py_function, which takes a python function which manipulates tf eager
(pid=7715, ip=192.168.8.55)     tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
(pid=7715, ip=192.168.8.55)     an ndarray (just call tensor.numpy()) but having access to eager tensors
(pid=7715, ip=192.168.8.55)     means `tf.py_function`s can use accelerators such as G

(pid=7715, ip=192.168.8.55) Restoring from /mnt/Ressources/Andres/Temp_active/runs/al_cifar_AL_42/checkpoint/epoch34.ckpt-34


 98% 1014/1037 [01:44<00:02,  9.84it/s]0m 
 98% 1015/1037 [01:44<00:02,  9.84it/s]0m 
 98% 1017/1037 [01:44<00:02,  9.82it/s]0m 
 98% 1018/1037 [01:44<00:02,  9.44it/s]0m 
 98% 1020/1037 [01:44<00:01,  9.74it/s]0m 
 99% 1022/1037 [01:44<00:01, 10.01it/s]0m 
 99% 1024/1037 [01:45<00:01, 10.42it/s]0m 
(pid=7715, ip=192.168.8.55) 2020-12-01 17:54:45.017598: I tensorflow/stream_executor/dso_loader.cc:152] successfully opened CUDA library libcublas.so.10.0 locally
 99% 1026/1037 [01:45<00:01, 10.44it/s]0m 
 99% 1028/1037 [01:45<00:00, 10.56it/s]0m 
 99% 1030/1037 [01:45<00:00, 10.51it/s]0m 
100% 1032/1037 [01:45<00:00,  9.60it/s]0m 
100% 1034/1037 [01:46<00:00,  9.97it/s]0m 
100% 1036/1037 [01:46<00:00, 10.69it/s]0m 
100% 1037/1037 [01:46<00:00,  9.75it/s]0m 
(pid=7715, ip=192.168.8.55) 
(pid=7715, ip=192.168.8.55) wandb: Waiting for W&B process to finish, PID 7738
(pid=7715, ip=192.168.8.55) wandb: Program ended successfully.
(pid=7715, ip=192.168.8.55) wandb: Run summary:
(pid=7715, ip=19

## Get the global training step as progress

In [ ]:
progress_id = NetworkActor.get_progress.remote()
ray.get(progress_id)

## Stop the training and perform

In [ ]:
NetworkActor.stop_training.remote()

## Kill the training actor

In [ ]:
NetworkActor.__ray_terminate__.remote()